## Library

In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings(action="ignore")

DATA_PATH = "/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/"

## import data

In [2]:
df_train = pd.read_csv(DATA_PATH+"train.csv")
df_test = pd.read_csv(DATA_PATH+"test.csv")

In [3]:
df_train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [4]:
pd.set_option("display.max_colwidth", None) # 텍스트 데이터 생략 없이 전체 표기.
'''
여러 컬럼은 전달 시 dataframe 2차원이라서 두겹으로 작성.
단일 컬럼은 series라서 한겹.
'''
df_train.sample(30)[['id', 'target', 'comment_text']]

,id,target,comment_text
1136461,5504421,0.200000,"""I don't know why you have 3""disagrees"" (as I write this).....you are 100% correct and this is entirely verifiable.\n\nThere are 15 replies right now....in this unscientific mini-sample, that is 20% of the respondents who don't care about the truth and can't be bothered to do a simple Google search to verify information they are being told. Interesting..... :)""\n-\n-\n-\nWhen the Father of Lies is the cook, His followers develop a taste for falsehood.\n\nBesides, it's not about truth or falsehood, it's one about the Schadenfreude in seeing ""libs' heads 'sploding!!!"""
146248,420916,0.200000,"I dearly wish the most offensive and the most dangerous thing about Mr. Trump was the brand of gum he chews. Unfortunately, as documented by years of his actions and comments and, especially, his ongoing actions and comments, Mr. Trump has presented and continues to present himself as the most unfit (unfit intellectually, unfit morally, unstable emotionally and IMO would be diagnosed as a sociopath) candidate for President in our history. He's not just a ""loose cannon"", he's an extremely flawed ""loose cannon."" \n\nIf you feel my concerns are delusional, please study his personal history, study his business history, read his comments over the years, review the videos of his actions, watch his acceptance speech at the RNC, listen to the critiques of those who have worked with him. view his interviews, even those with extremely sympathetic Fox News anchors, and decide for yourself. I'm hardly in a position to cause us to ""lose our country."" Elected President, Trump would be..... Gary"
551752,917975,0.400000,Why arent these people Canadian in the fisrt place. You want a business in Canada then you damn well better be filling every single job with Canadians and our government should see to this if they are doing their jobs. Surely ther are people out there to fill these jobs. Some just dont get the point of all this.
333113,649994,0.000000,"""Open wound."""
1275028,5672686,0.000000,"Lol, whatever helps you sleep at night.\n\nTell me lies, tell me sweet little lies...."
1406455,5837457,0.000000,KPMG provided bizarre reasons for terminating a 15 year old business relationship with the Guptas. My question to KPMG is that Why would anyone abandon clients because of “political interests”?
1488314,5941554,0.846154,What part of public land do idiots like you not understand?
1331139,5742007,0.000000,"They never think about that.\nTo them, the USA has a one-way license to interfere anywhere.\nFunny how we don't like it when Russia or China has the same attitude."
1015580,5359385,0.166667,"Trump can't fire Pence ... but I wonder if he has something else on Pence. Pence's poor judgment and lack of self-respect seem inexplicable unless he's under some form of duress. Pence's oath and his office require him to serve the United States; his performance thus far has been as lackey to and enabler of Donald Trump. Trump is disgracing the office of president; in a different way, Pence is disgracing the office of vice-president."
982264,5319526,0.000000,Somehow it's not reassuring that the problem luggage will be checked-in rather than carry on.


In [5]:
df_train.shape

(1804874, 45)

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1804874 entries, 0 to 1804873
Data columns (total 45 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   id                                   int64  
 1   target                               float64
 2   comment_text                         object 
 3   severe_toxicity                      float64
 4   obscene                              float64
 5   identity_attack                      float64
 6   insult                               float64
 7   threat                               float64
 8   asian                                float64
 9   atheist                              float64
 10  bisexual                             float64
 11  black                                float64
 12  buddhist                             float64
 13  christian                            float64
 14  female                               float64
 15  heterosexual                    

In [7]:
def missing_count(df: pd.DataFrame) -> pd.Series:
    """
    각 컬럼별 결측치 개수를 반환하는 함수
    df.count() 기반으로 계산
    """
    n_rows = len(df)
    return n_rows - df.count()
    # df.count()는 non-null만 세기에 isnull.sum()보다 빠름

missing = missing_count(df_train)
print(missing)

id                                           0
target                                       0
comment_text                                 3
severe_toxicity                              0
obscene                                      0
identity_attack                              0
insult                                       0
threat                                       0
asian                                  1399744
atheist                                1399744
bisexual                               1399744
black                                  1399744
buddhist                               1399744
christian                              1399744
female                                 1399744
heterosexual                           1399744
hindu                                  1399744
homosexual_gay_or_lesbian              1399744
intellectual_or_learning_disability    1399744
jewish                                 1399744
latino                                 1399744
male         

In [8]:
# 문자형 unique 확인
df_train.describe(include="object").T

,count,unique,top,freq
comment_text,1804871,1780822,Well said.,184
created_date,1804874,1804362,2015-10-13 18:40:35.757707+00,4
rating,1804874,2,approved,1684758


In [9]:
# 수치형 
df_train.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
id,1804874.0,3738433.79,2445186.76,59848.0,796975.25,5223774.5,5769854.50,6334010.00
target,1804874.0,0.10,0.20,0.0,0.00,0.0,0.17,1.00
severe_toxicity,1804874.0,0.00,0.02,0.0,0.00,0.0,0.00,1.00
obscene,1804874.0,0.01,0.06,0.0,0.00,0.0,0.00,1.00
identity_attack,1804874.0,0.02,0.08,0.0,0.00,0.0,0.00,1.00
insult,1804874.0,0.08,0.18,0.0,0.00,0.0,0.09,1.00
threat,1804874.0,0.01,0.05,0.0,0.00,0.0,0.00,1.00
asian,405130.0,0.01,0.09,0.0,0.00,0.0,0.00,1.00
atheist,405130.0,0.00,0.05,0.0,0.00,0.0,0.00,1.00
bisexual,405130.0,0.00,0.03,0.0,0.00,0.0,0.00,1.00
